![Noteable.ac.uk Banner](https://github.com/jstix/mr-noteable/blob/master/Banner%20image/1500x500.jfif?raw=true)

# Noteable Data Visualization Example

## Anthromes - tracking human impact in an ecological region


### Instructions: “Run” the cells to see the graphs
Click “Cell” and select “Run All”. <br>This will import the data and run all the code, so you can see this notebook's data visualizations (scroll to the top after you’ve run the cells). <br>

**You don’t need to do any coding**.


## Goal
Our goal is to show the gradual then sudden change in land-use patterns from pre-history to modern day.

We will use a "stacked line graph" showing the landmasses of different land-use categories varying from urbanized to wild lands.

### 1. Question

Have you ever wondered how human civilizations have used the Earth's land?

For instance, how has farming and industrialization changed land-use patterns. Can we see this in a dataset of land use?


### 2. Gather
The code below will import the Python programming libraries we need to gather and organize the data to answer our question.

In [9]:
import pandas as pd #the pandas library is used to organize our data into tables known as "pandas dataframes"
import os #used to create OS agnostic file paths
from plotly.subplots import make_subplots #used to create our interactive plots
import plotly.graph_objects as go #used to create our interactive plots

### About our data

Models of historic land use can provide evidence for future scenarios of land-use change. Our dataset is from the Netherlands Environmental Assessment Agency's [History Database of the Global Environment](https://themasites.pbl.nl/tridion/en/themasites/hyde/). The [particular dataset](https://dataportaal.pbl.nl/downloads/HYDE/HYDE3.2/) we are using is one on anthromes since 10000 BC. Anthromes, or [Anthropogenic Biomes](https://en.wikipedia.org/wiki/Anthropogenic_biome), are a classification system for the Earth's surface on the basis of how human-altered the land is.  

The dataset breaks down into 6 main categories. These are described below.

| Name              | Description                                                                  |
|-------------------|------------------------------------------------------------------------------|
| Dense Settlements | Urban and other nonagricultural dense settlements                            |
| Villages          | Densely populated agricultural settlements                                   |
| Croplands         | Lands used mainly for annual crops                                           |
| Rangelands        | Lands used for pasture and livestock grazing                                 |
| Seminatural       | Inhabited lands with minor use for permanent agriculture and settlements     |
| Wild              | Lands without human populations or substantial land use                      |

It further uses many different time intervals for its measurements. These are described in the table below.

| Interval  | Date Range    |
|-----------|---------------|
| Millennia | 10000-1000 BCE|
| Century   |    0-1600     |
| Decade    |    1700-2000  |
| Year      |    2001-2017  |

Much of the modern data is obtained by recorded  measurements, but the earlier data points were created using historical and archeological methods. [This paper](https://www.mdpi.com/2073-445X/9/5/129) describes the dataset in greater detail.

### Importing our data
This next block of code will read the data file and save it in a dataframe named `df`.

In [10]:
path = os.path.join("data", "/home/jovyan/anthromes_summary.txt") #create an OS agnostic file path
df = pd.read_csv(path, delim_whitespace=True) # create df
df = df.set_index("Class_id") # set index of df

In [11]:
df.head() # display df

,-10000,-9000,-8000,-7000,-6000,-5000,-4000,-3000,-2000,-1000,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Class_id,,,,,,,,,,,,,,,,,,,,,
11,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,...,6.493338e+05,6.592988e+05,6.695398e+05,6.791712e+05,6.913848e+05,7.028646e+05,7.127421e+05,7.227881e+05,7.326726e+05,7.434778e+05
12,0.0,0.0,60.2029,60.2029,44.1170,44.1170,277.7667,602.0583,1360.16560,4835.98692,...,1.127953e+06,1.135130e+06,1.139931e+06,1.145093e+06,1.149549e+06,1.157465e+06,1.166687e+06,1.174036e+06,1.180871e+06,1.189352e+06
21,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000,148.27670,...,1.544662e+06,1.513149e+06,1.533354e+06,1.548495e+06,1.541316e+06,1.563751e+06,1.571130e+06,1.580805e+06,1.587834e+06,1.597813e+06
22,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,805.98980,2210.69690,...,1.789168e+06,1.833284e+06,1.864894e+06,1.880495e+06,1.905639e+06,1.922416e+06,1.948258e+06,1.975543e+06,1.999000e+06,2.025359e+06
23,0.0,0.0,0.0000,0.0000,60.2029,177.5596,320.8751,320.8751,936.50707,2470.59194,...,5.765092e+06,5.800291e+06,5.795371e+06,5.811165e+06,5.843118e+06,5.856475e+06,5.886492e+06,5.915619e+06,5.951228e+06,5.978826e+06


### Comment on the data

The dataframe above has the categories we want encoded in the index and broken down into granular sub categories and has no separation between data points separated by a millennia or a year.

### 3. Organize

The code below will reorganize our code and break it into meaningful subparts so we can create our plots. 

In [12]:
# The lines below sum subcategories into the 6 main catagories we described earlier

df.loc["Dense Settlements"] = df.loc[[11, 12]].sum()
df.loc["Villages"] = df.loc[[21, 22, 23, 24]].sum()
df.loc["Croplands"] = df.loc[[31, 32, 33, 34]].sum()
df.loc["Rangeland"] = df.loc[[41, 42, 43]].sum()
df.loc["Seminatural"] = df.loc[[51, 52, 53, 54]].sum()
df.loc["Wild"] = df.loc[[61, 62, 63]].sum()

In [13]:
df = df[-6:] # remove all the original granular data
df #display the new df

,-10000,-9000,-8000,-7000,-6000,-5000,-4000,-3000,-2000,-1000,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Class_id,,,,,,,,,,,,,,,,,,,,,
Dense Settlements,0.000000e+00,0.000000e+00,6.020290e+01,6.020290e+01,4.411700e+01,4.411700e+01,2.777667e+02,6.020583e+02,1.360166e+03,4.835987e+03,...,1.777287e+06,1.794429e+06,1.809471e+06,1.824264e+06,1.840934e+06,1.860330e+06,1.879429e+06,1.896824e+06,1.913543e+06,1.932830e+06
Villages,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.020290e+01,1.775596e+02,3.208751e+02,3.208751e+02,1.742497e+03,4.829566e+03,...,9.309398e+06,9.362151e+06,9.410116e+06,9.462959e+06,9.510986e+06,9.564321e+06,9.632336e+06,9.699635e+06,9.767670e+06,9.836441e+06
Croplands,0.000000e+00,0.000000e+00,0.000000e+00,4.324020e+00,5.844304e+02,9.933780e+03,1.621030e+05,2.768684e+05,4.158473e+05,8.822548e+05,...,2.696107e+07,2.692868e+07,2.687617e+07,2.686427e+07,2.691132e+07,2.690918e+07,2.693846e+07,2.695964e+07,2.697589e+07,2.698622e+07
Rangeland,0.000000e+00,0.000000e+00,1.639491e+01,2.985281e+01,3.640676e+01,4.431940e+01,4.869301e+01,5.370604e+01,5.250286e+02,1.000303e+06,...,2.888293e+07,2.885079e+07,2.882244e+07,2.870616e+07,2.862574e+07,2.856639e+07,2.847606e+07,2.839602e+07,2.831254e+07,2.823643e+07
Seminatural,7.873286e+07,8.005476e+07,8.129208e+07,8.242849e+07,8.349912e+07,8.425942e+07,8.485423e+07,8.530539e+07,8.608326e+07,8.537321e+07,...,3.047448e+07,3.046739e+07,3.046696e+07,3.053879e+07,3.050795e+07,3.056479e+07,3.054351e+07,3.051824e+07,3.049052e+07,3.046637e+07
Wild,5.307118e+07,5.174928e+07,5.051188e+07,4.937545e+07,4.830419e+07,4.753442e+07,4.678706e+07,4.622080e+07,4.530131e+07,4.453861e+07,...,3.439888e+07,3.440060e+07,3.441888e+07,3.440760e+07,3.440710e+07,3.433903e+07,3.433425e+07,3.433368e+07,3.434388e+07,3.434575e+07


Next, we'll break the df down into 4 sub dataframes based on the time interval between data points.

In [14]:
#these commands create our 4 new dataframes based on time interval
df_millennia = df.iloc[:, :11]
df_centuries = df.iloc[:, 10:28]
df_decades = df.iloc[:, 27:58]
df_years = df.iloc[:, 57:]

### 4. Explore

To explore our dataset, we'll create 4 plots, one for each of the dataframes we created above. If we tried to create one plot with a linear time scale, the more frequent data points would be squished together and it would be hard to see many details.

In [17]:
#this function adds traces to a plotly fig object

def add_traces(fig, df, row, col, traces, showlegend=False):
    colors = ['grey', 'teal', 'yellow', 'brown', 'blue', 'green']
    i = 0;
    x = df.columns
    for trace in traces:
        fig.add_trace(go.Scatter(
        x=x,
        y=df.loc[trace].array,
        legendgroup="group" + str(i),
        name = trace,
        mode='lines+markers',
        stackgroup='one',
        showlegend=showlegend,
        line_color=colors[i]),
                      row=row, 
                      col=col)
        i = i + 1
    return fig

In [1]:
anthromes = ["Dense Settlements", "Villages", "Croplands", "Rangeland", "Seminatural", "Wild"]
#create fig with subplots
fig = make_subplots(rows=2, cols=2,
                   subplot_titles=['Millennia', 'Centuries', 'Decades', 'Years'],
                   x_title="Year (negative dates are BCE)",
                   y_title="km squared")

#use the add_traces function to create the four plots
fig = add_traces(fig, df_millennia, 1, 1, anthromes, True)
fig = add_traces(fig, df_centuries, 1, 2, anthromes)
fig = add_traces(fig, df_decades, 2, 1, anthromes)
fig = add_traces(fig, df_years, 2, 2, anthromes)

#add title
fig.update_layout(title_text='Changes in Anthromes Across Time')
#show plot
fig.show()

NameError: name 'make_subplots' is not defined

### 5. Interpret
Below, we will discuss the plot we created and how to examine it.

First, when looking at these plots we must ensure we are aware of the varying interval between points. The top left plot(where data points are seperated by 1000 year intervals), 10000 BCE to 0, is incredibly stable with very little land devoted to settlements or agriculture. Between 0 and 1700 we see more change than in the previous 10000 years. The plot with the most dynamic elements is the one between 1700 and 2000. Here, we see incredible shifting away from seminatural lands to agricultural lands and the emergence of more villages around the turn of the last century. Finally, between 2000 and 2017 we note little change.

To help us see this change more clearly, the creators of this data created world maps showing land use. Look at the change between 10000 BCE and 2017.

![10000BCE](https://github.com/jstix/mr-noteable/blob/master/Notebook%20images%20-%20Anthromes/anthromes12k_010000_10000BC.png?raw=true)
![2017](https://github.com/jstix/mr-noteable/blob/master/Notebook%20images%20-%20Anthromes/anthromes12k_022017_02017AD.png?raw=true)

### 6. Communicate
#### Reflect on the interpretation
What can data reveal about changes in land use over time?

**Continuity and change**
- What has stayed the same and what has changed across time?

**Cause and effect**
- What human activities and natural phenomena affect change in land use?
- How can human activities contribute to solutions to land-use change?

**Ethics**
- How can personal and societal choices impact change?
- How might land-use change impact society or the economy?

![Noteable license](https://github.com/jstix/mr-noteable/blob/master/Banner%20image/Screenshot%202021-03-05%20115453.png?raw=true)